In [4]:
import pandas as pd
import haversine as hs
from datetime import datetime
import googlemaps

In [5]:
establishments = pd.read_excel('../../_data/cajamarca_data_helth_estb.xlsx')
data_centroids = pd.read_excel('../../_data/peru_districts_centroids.xlsx')

In [6]:
centroids = data_centroids.query("NOMBDEP == 'CAJAMARCA'")

In [7]:
# Algorithm which searches for the nearest centroid to an establishment
def get_closest_centroid(establishment):
    distances = {
        tuple(centroid[['Centroid_Latitude', 'Centroid_Longitude']]):
        hs.haversine(
            (establishment['latitud'], establishment['longitud']),
            tuple(centroid[['Centroid_Latitude', 'Centroid_Longitude']])
        ) for i, centroid in centroids.iterrows()
    }
    return min(distances, key=distances.get)

In [8]:
nearest_centroids = list(
    map(
        lambda establishment: get_closest_centroid(establishment[1]),
        establishments.iterrows()
    )
)

In [9]:
establishments['Dist_Centroid_Latitude'] = list(map(lambda x: x[0], nearest_centroids))
establishments['Dist_Centroid_Longitude'] = list(map(lambda x: x[1], nearest_centroids))

In [10]:
gmaps = googlemaps.Client( key = 'YOUR KEY HERE' )

ValueError: Invalid API key provided.

In [ ]:
test = establishments.head(1)

time = datetime(2023, 3, 13, 16, 0, 0, 0)

directions = gmaps.directions(
    (test['latitud'].values[0], test['longitud'].values[0]),
    (test['Dist_Centroid_Latitude'].values[0], test['Dist_Centroid_Longitude'].values[0]),
    departure_time=time,
    mode='transit',
)

directions